# Predicting Movie Performance

I have compiled data for all the movies from 2011-2016 that had an opening US weekend box office total of $10m or more. We have some limited data points for each movie including genre, creative source, and production budget. Given the data for films from 2011-2015, can we predict the results of movies in 2016?

We'll be using a variety of decision tree techniques, including random forests and boosted trees.

In [ ]:
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
import pydot
from Ipython.display import Image
from sklearn import tree

## Data Preparation
Here we'll be using pandas (http://pandas.pydata.org/) to import our data sets and ready our data for analysis. I've made sure that the data is already pretty normalized to make things easy on us.

In [9]:
train = pd.read_csv('train.csv',parse_dates=[0])
trainY = train['Infl. Adj. Dom. Box Office']
del train['Infl. Adj. Dom. Box Office']

test = pd.read_csv('test.csv',parse_dates=[0])
testY = test['Infl. Adj. Dom. Box Office']
del test['Infl. Adj. Dom. Box Office']

train.head()
#trainY.head()

,Released,Title,Theatrical Distributor,Genre,Source,Production Method,Creative Type,Production Budget,Theatrical Engagements
0,2013-04-12,42,Warner Bros.,Drama,Based on Real Life Events,Live Action,Dramatization,40000000.0,21418.0
1,2013-08-02,2 Guns,Universal,Action,Based on Comic/Graphic Novel,Live Action,Contemporary Fiction,61000000.0,15505.0
2,2012-03-16,21 Jump Street,Sony Pictures,Comedy,Based on TV,Live Action,Contemporary Fiction,42000000.0,22744.0
3,2014-06-13,22 Jump Street,Sony Pictures,Comedy,Based on TV,Live Action,Contemporary Fiction,50000000.0,22821.0
4,2014-02-21,3 Days to Kill,Relativity,Thriller/Suspense,Original Screenplay,Live Action,Contemporary Fiction,28000000.0,11265.0


## Building Our First Decision Tree

In [ ]:
regressor = DecisionTreeRegressor()
regressor.fit(train,trainY)

dotfile = StringIO()
export_graphviz(regressor, out_file=dotfile)
pydot.graph_from_dot_data(dotfile.getvalue()).write_png("tree.png")

Image(filename='tree.png')